In [ ]:
import batoid
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_r_baffles.yaml")
wavelength = 622e-9

In [ ]:
# Need a function to project from CenterPlateAndSpiderTop to focal plane
# Trace a bunch of rays at the given field angle and then 2x2D interpolate.

In [ ]:
def corners(rect, thx=0, thy=0, height=0):
    x = rect.width/2 * np.array([-1, 1, 1, -1, -1])
    y = rect.height/2 * np.array([-1, -1, 1, 1, -1])
    sth, cth = np.sin(rect.theta), np.cos(rect.theta)
    x1 = x * cth - y * sth
    y1 = x * sth + y * cth
    x1 += rect.x
    y1 += rect.y
    if height != 0:
        vx, vy, vz = batoid.utils.fieldToDirCos(thx, thy)
        x1 += height * vx / vz
        y1 += height * vy / vz
    return x1, y1

In [ ]:
# Let's draw!
thx, thy = np.deg2rad([1.5, 1.5])
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
for rect in telescope["CenterPlateAndSpiderTop"].obscuration.items[:4]:
    axs[0].plot(*corners(rect), c='r', lw=1)
    axs[1].plot(*corners(rect, thx, thy, telescope["CenterPlateAndSpiderTop"].coordSys.origin[2]), c='r', lw=1)
for rect in telescope["UpperTEABaffleAndSpiderBottom"].obscuration.items[1:]:
    axs[0].plot(*corners(rect), c='r', lw=1)
    axs[1].plot(*corners(rect, thx, thy, telescope["UpperTEABaffleAndSpiderBottom"].coordSys.origin[2]), c='r', lw=1)
th = np.linspace(0, 2*np.pi, 1000)
for ax in axs:
    ax.plot(4.18*np.cos(th), 4.18*np.sin(th), lw=1, c='r')
    ax.plot(4.18*0.612*np.cos(th), 4.18*0.612*np.sin(th), lw=1, c='r')
    ax.set_aspect("equal")
plt.show()

In [ ]:
# For a single field angle, can model as 4 finite width but infinite length boxes.
# Unfortunately, the width is often smaller than a single projected pixel.
# So need to solve a fairly complicated problem of multiple partial obscurations.
# Especially complicated if RTP = 45 and slopes of lines in single rect may have opposite sign.
# Always cuts across annulus.  Don't worry about corners where annulus and spider both intersect.  Use minimum or something....

In [ ]:
# Spider centers move in x at rate
# vx/vz * height
# For heights: 8.618 and 7.418

In [ ]:
print(telescope["CenterPlateAndSpiderTop"].coordSys.origin[2])
print(telescope["UpperTEABaffleAndSpiderBottom"].coordSys.origin[2])

In [ ]:
# So width grows something like vr/vz dheight where dheight = 8.618 - 7.418 and vr =